# 개요
> 본 노트북 코드는 **통근 OD 데이터**를 산출하기 이전에 **100명**의 랜덤 샘플을 두고 몇 퍼센트의 통근 패턴을 감지할 수 있는 지를 기반으로 로직의 합당성을 판단하는 로직임

In [ ]:
import duckdb as duck
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from scipy.spatial import Voronoi
from shapely.geometry import Polygon
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
# duckdb db 불러오기
con = duck.connect(database='myanalysis.db', read_only=False)
con

In [ ]:
# 정류장 정보 들고 오기
station = pd.read_csv('import_data/TB_KTS_STTN/202406/TB_KTS_STTN_20240603.csv')
station07 = pd.read_csv('import_data/TB_KTS_STTN/202407/TB_KTS_STTN_20240701.csv')

# 1. 2024년 6월 샘플 데이터 수집

In [ ]:
# 평일만 불러오기- 2024년 6월
dates = pd.date_range(start = '2024-06-01', end = '2024-06-30', freq='D')
weekday = dates[dates.weekday < 5]
weekday_strs = weekday.strftime('%Y%m%d').to_list()
card_sets = []

In [ ]:
q = f'''
    SELECT DISTINCT 가상카드번호
    FROM read_csv('import_data/TB_KTS_DWTCD_METROPOLITAN/202406/TB_KTS_DWTCD_METROPOLITAN_20240603.csv', columns ={{'가상카드번호':'VARCHAR'}})
    '''
cards = set(con.execute(q).fetchdf()['가상카드번호'])
sampled = random.sample(list(cards),100)
print(len(sampled))

In [ ]:
sample_ids = pd.DataFrame(sampled)[0].str.split(',').str[2]
sample_ids = sample_ids.to_list()
sample_ids

In [ ]:
# db에 insert
con.execute("""CREATE OR REPLACE TEMP TABLE cardlist
                (card_id VARCHAR)""")
for sample_id in sample_ids:
    con.execute("INSERT INTO cardlist VALUES (?)", [sample_id])

In [ ]:
# 100명에 대한 6월 한달 데이터 가져오기
dfs = []
single_card = sampled[0]
for day in weekday_strs:
    q = f'''
    SELECT *
    FROM read_csv('import_data/TB_KTS_DWTCD_METROPOLITAN/202406/TB_KTS_DWTCD_METROPOLITAN_{day}.csv', types= {{'정산사노선ID':'VARCHAR', '가상카드번호':'VARCHAR'}})
    WHERE 가상카드번호 IN (SELECT card_id FROM cardlist)
    '''
    df = con.query(q).to_df()
    dfs.append(df)

In [ ]:
full_month_df = pd.concat(dfs, ignore_index=True)
# full_month_df.columns = ['운행일자', '정산사ID', '가상카드번호', '정산지역코드', '카드구분코드', '정산사차량ID', '교통수단코드',
#        '정산사노선ID', '승차일시', '정산사승차정류장ID', '하차일시', '정산사하차정류장ID', '트랜잭션ID', '환승건수',
#        '이용자유형코드(시스템)', '이용자수', '이용거리', '탑승시간']
print(full_month_df.가상카드번호.nunique())
full_month_df

## 1.1. 24년 6월 샘플 데이터 저장 & 불러오기

In [ ]:
# full_month_df.to_csv('sampled_202406.csv')
full_month_df = pd.read_csv('sampled_202406.csv')

# 2. 목적통행으로 집계

In [ ]:
# 교통수단구분 & 정산지역코드 타입 정리
full_month_df['교통수단구분'] = full_month_df['교통수단코드'].apply(lambda x: 'T' if (199<x & x<300) else 'B')
full_month_df.정산지역코드 = full_month_df.정산지역코드.astype(str)

In [ ]:
# 집계 방식 정의

agg_dict = {**{x : 'first' for x in ['정산사ID', '이용자유형코드(시스템)']},
            **{x : 'sum' for x in ['이용거리', '탑승시간']},
           **{x: 'first' for x in ['정산사승차정류장ID', '승차일시']},
           **{x: 'last' for x in ['정산사하차정류장ID', '하차일시']}}
agg_dict['환승건수'] = 'max'
# multi-lines
multi_lines = {}
multi_lines['승차정산지역코드'] = ('정산지역코드', 'first')
multi_lines['하차정산지역코드'] = ('정산지역코드', 'last')
multi_lines['승차교통수단구분'] = ('교통수단구분', 'first')
multi_lines['하차교통수단구분'] = ('교통수단구분', 'last')
multi_lines['승차노선ID'] = ('정산사노선ID', 'first')
multi_lines['하차노선ID'] = ('정산사노선ID', 'last')

print(agg_dict)
multi_lines

In [ ]:
# 목적통행으로 집계  4037 -> 2614건으로
full_month_df.sort_values(['운행일자', '승차일시'], inplace=True)
main_trip = full_month_df.groupby(['운행일자', '가상카드번호', '트랜잭션ID']).agg(agg_dict).reset_index()
support_trip = full_month_df.groupby(['운행일자', '가상카드번호', '트랜잭션ID']).agg(**multi_lines).reset_index()
linked_202406_100 = pd.concat([main_trip, support_trip.iloc[:, 2:]], axis=1)
linked_202406_100

In [ ]:
# 가상카드번호에 대한 일별 목적 통행량에 대한 기초통계
daily_counts = linked_202406_100.groupby(['운행일자', '가상카드번호']).size().reset_index(name= 'day_counts')
daily_counts.day_counts.value_counts().reset_index().sort_values('day_counts')

# 3. 케이스 확인

In [ ]:
# 분석을 위한 데이터 전처리
# 1. 정류장ID -> int
linked_202406_100['정산사승차정류장ID'] = pd.to_numeric(linked_202406_100['정산사승차정류장ID'], errors='coerce')
linked_202406_100['정산사하차정류장ID'] = pd.to_numeric(linked_202406_100['정산사하차정류장ID'], errors='coerce')

# 2. 데이터프레임에 요일 붙이기
linked_202406_100['운행일자'] = pd.to_datetime(linked_202406_100['운행일자'].astype(str), format='%Y%m%d')
linked_202406_100['요일'] = linked_202406_100['운행일자'].dt.day_name()

# 3. 현충일(공휴일) 제외
linked_202406_100 = linked_202406_100[~(linked_202406_100.운행일자 =='2024-06-06')]

In [ ]:
linked_202406_100.가상카드번호.unique()

In [ ]:
linked_202406_100['이용자유형코드(시스템)'].value_counts()
# 1	일반인
# 2	어린이
# 3	청소년
# 4	경로
# 5	장애인
# 6	국가유공자
# 7	외국인
# 8	기타

# 4. 케이스 분류 로직 실행

## 4.1. 지하철 기준 보로노이 경계 생성

In [ ]:
# 3:30-4:00 1차 시도
# 4:00-5:40

# 지하철 정류장 보로노이 패턴 생성
subway = station[station['교통수단구분'] == 'T']
# subway[subway.duplicated(subset=['지역코드', '정류장명칭'], keep=False)].sort_values('정류장명칭') # 1158개 중 230개가 중복, 즉 115개 드랍 가능
# 호선이 다르지만 이름은 같은 경우 -> 하나의 좌표로 생성하도록
subway_no_dupe = subway.drop_duplicates(subset=['지역코드', '정류장명칭'], keep='first')

subway_pts = np.vstack([subway_no_dupe.정류장GPSX좌표, subway_no_dupe.정류장GPSY좌표]).T

vor_subway = Voronoi(subway_pts)
vor_subway

polys = []
for region in vor_subway.regions:
    if not region or -1 in region:
        continue
    verts = [vor_subway.vertices[i] for i in region]
    poly = Polygon(verts)
    polys.append(poly)
voronoi_subway = gpd.GeoDataFrame(geometry=polys, crs = 'EPSG:5179')
voronoi_subway = voronoi_subway.reset_index().rename(columns={'index':'cluster_id'})
voronoi_subway.to_csv('voronoi_subway.csv')
voronoi_subway

In [ ]:
# station에 클러스터ID 붙이기
# 지방에는 지하철이 많이 없어서 NULL 값 발생
station_gdf = gpd.GeoDataFrame(station, geometry = gpd.points_from_xy(station.정류장GPSX좌표, station.정류장GPSY좌표))
station_labeled = station_gdf.sjoin(voronoi_subway, how='left', predicate='within').drop('index_right', axis=1)
print(len(station_labeled))
station_labeled.isnull().sum()
# station_labeled.to_csv('station_labeled.csv')

## 4.2. 최빈값 로직 실행

### 4.2.1. 클러스터ID 붙이기

In [ ]:
# 클러스터id 붙이기
cluster_202406_b = linked_202406_100.merge(station_labeled[['정류장ID', '정류장명칭', '지역코드','교통수단구분', '정류장GPSX좌표', '정류장GPSY좌표', 'cluster_id']],
                               how='left',
                                left_on =['승차정산지역코드','승차교통수단구분', '정산사승차정류장ID'],
                                right_on = ['지역코드','교통수단구분', '정류장ID']).drop(columns=['정류장ID', '지역코드', '교통수단구분']).rename(columns ={'cluster_id':'승차클러스터ID',
                                                                                                                                   '정류장명칭':'승차정류장명칭',
                                                                                                                                            '정류장GPSX좌표':'승차정류장_위도',
                                                                                                                                 '정류장GPSY좌표':'승차정류장_경도'})
# test._merge.value_counts()
cluster_202406 = cluster_202406_b.merge(station_labeled[['정류장ID', '정류장명칭', '지역코드','교통수단구분', '정류장GPSX좌표', '정류장GPSY좌표', 'cluster_id']],
                               how='left', left_on =['하차정산지역코드','하차교통수단구분', '정산사하차정류장ID'],
                               right_on = ['지역코드','교통수단구분', '정류장ID']).drop(columns=['정류장ID', '지역코드', '교통수단구분']).rename(columns ={'cluster_id':'하차클러스터ID',
                                                                                                                                   '정류장명칭':'하차정류장명칭',
                                                                                                                                            '정류장GPSX좌표':'하차정류장_위도',
                                                                                                                                 '정류장GPSY좌표':'하차정류장_경도'})
# test._merge.value_counts()
# 승하차일시 datetime 형식으로 변경
cluster_202406['승차일시'] = pd.to_datetime(cluster_202406['승차일시'], format ='%Y%m%d%H%M%S')
cluster_202406['하차일시'] = pd.to_datetime(cluster_202406['하차일시'], format ='%Y%m%d%H%M%S')
cluster_202406 = cluster_202406.sort_values('승차일시')
cluster_202406.가상카드번호.unique()

In [ ]:
station[station.정류장ID == 	1270	]

In [ ]:
# 케이스: 전체 통행건수가 적으며 랜덤한 이동. 8 달에 10번 미만
# 케이스: 통행건수가 많으나 목적지나 시간이 랜덤한 경우 5
# 케이스4: 12~16시 업무지에서 출발하는 케이스라 업무지가 산출되지 않는 케이스 5
# 케이스3: 일반적 통근패턴이나 내리는 곳이 거리가 있는 경우: ex. 서원역(주거지), 신림역 등 번갈아가면서 이용 1
# 케이스5: 두개의 출근지를 가짐- 하루는 홍대, 하루는 면목동으로 가는 경우 업무지 빈도값 불일치함 3
# 케이스6: 비추적 이동패턴이 섞인 경우- 주거지만 추정됨 1
# 케이스8: 하차정류장 Null값  1
# 케이스9: 귀가만 일정하게 하는 패턴 2
# 케이스10: 동네 방황형 2: 이용거리 중위값 1500m 미만
test_1 = cluster_202406[cluster_202406['가상카드번호'] == 'Yalk8Ur29Q98R017C2lxQtlgoiuv+cm8EHgSfN6NMb4=']
test_1.이용거리.median()

In [ ]:
test_1.value_counts(['승차클러스터ID', '하차클러스터ID'])

In [ ]:
# 주거지 테스트 오전 승차 정류장최빈값, 오후 하차 정류장 최빈값
print('<06~12시 기준 출발 정류장 빈도>', test_1[(6<=test_1['승차일시'].dt.hour)&(test_1['승차일시'].dt.hour<12)].승차클러스터ID.value_counts(),'\n')
print('<16~24시 기준 도착 정류장 빈도>', test_1[(16<=test_1['하차일시'].dt.hour)&(test_1['하차일시'].dt.hour<24)].하차클러스터ID.value_counts(),'\n')

# 업무지 테스트 오전 하차 정류장 최빈값, 오후 승차 정류장 최빈값
print('<06~12시 기준 도착 정류장 빈도>', test_1[(6<=test_1['하차일시'].dt.hour)&(test_1['하차일시'].dt.hour<12)].하차클러스터ID.value_counts(),'\n')
print('<16~24시 기준 출발 정류장 빈도>', test_1[(16<=test_1['승차일시'].dt.hour)&(test_1['승차일시'].dt.hour<24)].승차클러스터ID.value_counts())
# test_1[(test_1['승차일시'].dt.hour>6)&(test_1['승차일시'].dt.hour<12)].승차클러스터ID.mode()

In [ ]:
# # 개인 기준으로 최빈값 도출해보기
# def commuting_mode(df):
#     # df 최빈값 일치할 시에만
#     live_cluster = None
#     work_cluster = None
#     # 주거지 산출 - 오전 승차&오후 하차
#     board_am_mask = (6<=df['승차일시'].dt.hour)&(df['승차일시'].dt.hour<12)
#     alight_pm_mask = (16<=df['하차일시'].dt.hour)&(df['하차일시'].dt.hour<24)

#     # 업무지 산출 - 오전 하차&오후 승차
#     alight_am_mask = (6<=df['하차일시'].dt.hour)&(df['하차일시'].dt.hour<12)
#     board_pm_mask = (16<=df['승차일시'].dt.hour)&(df['승차일시'].dt.hour<24)


#     # 예상 주거지 로직 - 오전 승차 & 오후 하차 클러스터 최빈값 일치유무
#     try:
#         if df[board_am_mask].승차클러스터ID.mode().iloc[0] == df[alight_pm_mask].하차클러스터ID.mode().iloc[0]:
#             live_cluster = df[board_am_mask].승차클러스터ID.mode().iloc[0]
#     except Exception as e:
#         None

#     # 예상 업무지 로직 - 최빈값 일치유무
#     try:
#         if df[board_pm_mask].승차클러스터ID.mode().iloc[0] == df[alight_am_mask].하차클러스터ID.mode().iloc[0]:
#             work_cluster = df[alight_am_mask].하차클러스터ID.mode().iloc[0]
#     except Exception as e:
#         None

#     return pd.DataFrame({
#     'card_id':[df.iloc[0]['가상카드번호']],
#     'home_cluster':[live_cluster],
#     'work_cluster':[work_cluster],
#     'under_10': len(df)<10})

In [ ]:
# # 함수 적용
# test_cluster = cluster_202406.groupby('가상카드번호').apply(commuting_mode).reset_index(drop=True)
# # cluster_202406.groupby('가상카드번호').apply(test_mode).reset_index(drop=True)
# commute_cluster = test_cluster[(~test_cluster.home_cluster.isna())&(~test_cluster.work_cluster.isna())]
# test_cards = list(commute_cluster.card_id.unique())
# print('주거&퇴근클러스터 존재하는 인원 수: ', len(commute_cluster))

In [ ]:
# 68인의 미분류 케이스
# test_cluster[test_cluster.card_id == '3PXKELFhzI5unFXoRWFAlnI4mehBy21RMEtQxrEiK94=']
# test_cluster
# set(cluster_202406.가상카드번호.unique()) - set(test_cards)

In [ ]:
# 개인 기준으로 최빈값 도출해보기
# 평균 이동시간, 이동거리


def commuting_mode(df):
    # df 최빈값 일치할 시에만
    live_cluster = None
    work_cluster = None
    # 주거지 산출 - 오전 승차&오후 하차
    board_am_mask = (6<=df['승차일시'].dt.hour)&(df['승차일시'].dt.hour<12)
    alight_pm_mask = (16<=df['하차일시'].dt.hour)&(df['하차일시'].dt.hour<24)

    # 업무지 산출 - 오전 하차&오후 승차
    alight_am_mask = (6<=df['하차일시'].dt.hour)&(df['하차일시'].dt.hour<12)
    board_pm_mask = (16<=df['승차일시'].dt.hour)&(df['승차일시'].dt.hour<24)


    # 예상 주거지 로직 - 오전 승차 & 오후 하차 클러스터 최빈값 일치유무
    try:
        if df[board_am_mask].승차클러스터ID.mode().iloc[0] == df[alight_pm_mask].하차클러스터ID.mode().iloc[0]:
            live_cluster = df[board_am_mask].승차클러스터ID.mode().iloc[0]
        else:
            return None
    except Exception as e:
        None

    # 예상 업무지 로직 - 최빈값 일치유무
    try:
        if df[board_pm_mask].승차클러스터ID.mode().iloc[0] == df[alight_am_mask].하차클러스터ID.mode().iloc[0]:
            work_cluster = df[alight_am_mask].하차클러스터ID.mode().iloc[0]
        else:
            return None
    except Exception as e:
        None

    # 예상 주거지 정류장 찾기 ( L-> W 출근 케이스만 필터하여 수행)
    sub_df = df[(df.승차클러스터ID == live_cluster)&(df.하차클러스터ID == work_cluster)]
    live_sub_df = sub_df.groupby(['승차정산지역코드', '승차교통수단구분', '정산사승차정류장ID', '승차정류장명칭',
                                 '하차정산지역코드', '하차교통수단구분', '정산사하차정류장ID', '하차정류장명칭',
                                  # '승차정류장_위도', '승차정류장_경도','하차정류장_위도', '하차정류장_경도'
                                 ]).agg(to_work_frequency =('정산사승차정류장ID', 'size'),
                                        average_to_work_travel_time = ('탑승시간', 'mean'), average_to_work_travel_distance = ('이용거리', 'mean')).reset_index().rename(columns=
                                                                                      {'정산사승차정류장ID': '주거지정류장ID',
                                                                                       '정산사하차정류장ID': '업무지정류장ID',
                                                                                      '승차정류장명칭': '주거지정류장명칭',
                                                                                      '하차정류장명칭': '업무지정류장명칭',
                                                                                      '승차교통수단구분': '출발교통수단구분',
                                                                                       '하차교통수단구분': '도착교통수단구분',
                                                                                      '승차정산지역코드':'출발정산지역코드',
                                                                                      '하차정산지역코드':'도착정산지역코드'})


    # 예상 업무지 정류장 찾기 ( L <- W 퇴근 케이스)
    sub_df = df[(df.승차클러스터ID == work_cluster)&(df.하차클러스터ID == live_cluster)]
    work_sub_df = sub_df.groupby(['승차정산지역코드', '승차교통수단구분', '정산사승차정류장ID', '승차정류장명칭',
                                   '하차정산지역코드', '하차교통수단구분', '정산사하차정류장ID', '하차정류장명칭',
                                  # '승차정류장_위도', '승차정류장_경도', '하차정류장_위도', '하차정류장_경도'
                                 ]).agg(to_live_frequency =('정산사하차정류장ID', 'size'),
                                        average_to_live_travel_time = ('탑승시간', 'mean'), average_to_live_travel_distance = ('이용거리', 'mean')).reset_index().rename(columns=
                                                                                      {'정산사승차정류장ID': '업무지정류장ID', # 업무지 -> 주거지로 출발지<-> 도착지, 등 순서를 바꿔서 적용
                                                                                       '정산사하차정류장ID': '주거지정류장ID', # Merge로 한 줄로 합치기 위함
                                                                                      '승차정류장명칭': '업무지정류장명칭',
                                                                                      '하차정류장명칭': '주거지정류장명칭',
                                                                                      '승차교통수단구분': '도착교통수단구분',
                                                                                       '하차교통수단구분': '출발교통수단구분',
                                                                                      '승차정산지역코드':'도착정산지역코드',
                                                                                      '하차정산지역코드':'출발정산지역코드'})
    live_work_df = live_sub_df.merge(work_sub_df, how = 'outer', on = ['주거지정류장ID', '업무지정류장ID',
                                                                      '출발정산지역코드', '출발교통수단구분','주거지정류장명칭',
                                                                     '도착정산지역코드', '도착교통수단구분', '업무지정류장명칭'])
    return live_work_df

In [ ]:
# 함수 적용
test_cluster = cluster_202406.groupby('가상카드번호').apply(commuting_mode).reset_index(drop=True)
test_cluster
test_cluster.to_csv('_output/sample_100_live_work.csv', encoding='euc-kr')

In [ ]:
# 정류장 좌표 정보 붙이기
test_100_station = test_cluster.merge(station[['지역코드', '교통수단구분', '정류장ID', '정류장명칭', '정류장GPSY좌표', '정류장GPSX좌표']], how='left',
                                      left_on = ['expect_home_station_id', '승차정산지역코드', '승차교통수단구분'],
                                       right_on = ['정류장ID', '지역코드', '교통수단구분'],
                                      indicator=True)
print(test_100_station._merge.value_counts())
test_100_station